In [1]:
!pip install langgraph-supervisor langchain-openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-1.0.5-py3-none-any.whl (471 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.
langchain-groq 0.3.8 requires langchain-core<1.0.0,>=0.3.75, but you have langchain-core 1.0.5 which is incompatible.
langchain-google-genai 2.0.10 requires langchain-core<0.4.0,>=0.3.37, but you have langchain-core 1.0.5 which is incompatible.


In [43]:
from langchain_openai import ChatOpenAI

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_groq import ChatGroq
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

# Initialize OpenAI model
model=  ChatGroq(api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")


In [3]:
!pip install ddgs

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install langchain langchain_community langgraph

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.5
    Uninstalling langchain-core-1.0.5:
      Successfully uninstalled langchain-core-1.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-supervisor 0.0.30 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-openai 1.0.3 requires langchain-core<2.0.0,>=1.0.2, but you have langchain-core 0.3.79 which is incompatible.


In [44]:
from langchain_community.tools import DuckDuckGoSearchRun

def search_duckduckgo(query: str):
    """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

# # Example usage
# result = search_duckduckgo("who is the president of the USA?")
# print(result)

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

In [45]:

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)


/tmp/ipykernel_1745499/777070536.py:4: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  math_agent = create_react_agent(


In [46]:
research_agent = create_react_agent(
    model=model,
    tools=[search_duckduckgo],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)


/tmp/ipykernel_1745499/1711993233.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  research_agent = create_react_agent(


In [47]:
# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

In [48]:
# Compile and run
app = workflow.compile()

result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is current news.and 5*26+30"
        }
    ]
})

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=search_duckduckgo [{"query": "current news"}]} </function>'}}

In [ ]:
result

{'messages': [HumanMessage(content='what is current news.and 5*26+30', additional_kwargs={}, response_metadata={}, id='27180258-fc02-449e-a3df-1a2fb10af11b'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 107, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-Ccraoh7tQPmWLBPTQuzvCIfLLjamr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='lc_run--e991492f-d8aa-44d3-b64c-83c73d7ee29a-0', tool_calls=[{'name': 'transfer_to_research_expert', 'args': {}, 'id': 'call_3IVais7kUmmcLHdX413Px6IM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 107, 'ou

In [ ]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is current news.and 5*26+30
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_3IVais7kUmmcLHdX413Px6IM)
 Call ID: call_3IVais7kUmmcLHdX413Px6IM
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

The current news includes updates from major sources like FOX News, CNN, AP News, NBC News, and CBS News covering topics such as U.S., world, entertainment, health, business, technology, and politics.

Regarding the calculation 5*26+30, the search results did not provide a direct answer, but the arithmetic expression evaluates to 5 times 26 plus 30. If you want, I can calculate it for you.
========

In [49]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is langgraph and 50*2*3"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=search_duckduckgo,{"query": "LangGraph applications in natural language processing and AI technology and how it can be used to drive innovation and improvement in various industries and what are the potential benefits and challenges of using LangGraph in real-world applications and how does it compare to other AI frameworks and tools and what are the future directions and potential developments of LangGraph and how will it impact the field of AI and NLP and what are the implications of using LangGraph in various industries and how will it change the way we approach problem-solving and decision-making and what are the potential risks and limitations of using LangGraph and how can they be addressed and what are the best practices for using Lang

In [13]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "who is the hero in the war2 movie"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

who is the hero in the war2 movie
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (call_5CCV9vdDQiucQGN05nI7k1fo)
 Call ID: call_5CCV9vdDQiucQGN05nI7k1fo
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

In the upcoming movie "War 2," the heroes are expected to be portrayed by Hrithik Roshan and Jr NTR. The film is directed by Ayan Mukerji and is part of the YRF Spy Universe.
================================== Ai Message ==================================
Name: research_expert

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (a17b74d6-6f95-43ca-9365-88e4c2dcd54b)
 Call ID

In [14]:
!pip install langgraph langgraph-checkpoint langgraph-checkpoint-sqlite


Defaulting to user installation because normal site-packages is not writeable
ERROR: Operation cancelled by user
^C


In [ ]:
from typing import Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

# --- Define State ---
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# --- Define LLMs (OpenAI instead of Anthropic) ---
research_llm = ChatOpenAI(model="gpt-4o-mini")
writer_llm = ChatOpenAI(model="gpt-4o-mini")

# --- Define Tools ---
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

# Bind tools to research agent
research_llm_with_tools = research_llm.bind_tools(tools)

# --- Define Agents ---
def research_agent(state: State):
    """Agent that decides if it needs to use a tool."""
    return {"messages": [research_llm_with_tools.invoke(state["messages"])]}

def writer_agent(state: State):
    """Agent that summarizes research results into a final answer."""
    return {"messages": [writer_llm.invoke(state["messages"])]}

# --- Add Nodes ---
graph_builder.add_node("research_agent", research_agent)
graph_builder.add_node("tools", ToolNode(tools=tools))
graph_builder.add_node("writer_agent", writer_agent)

# --- Add Edges ---
graph_builder.add_conditional_edges("research_agent", tools_condition)
graph_builder.add_edge("tools", "writer_agent")
graph_builder.add_edge("research_agent", "writer_agent")

# --- Set Entry & Finish Points ---
graph_builder.set_entry_point("research_agent")
graph_builder.set_finish_point("writer_agent")

# --- Compile Graph (NO memory/checkpointer) ---
graph = graph_builder.compile()

# --- Visualize Graph ---
try:
    display(Image(graph.get_graph().draw_mermaid_png()))  # PNG graph
except Exception as e:
    print("Mermaid PNG render failed:", e)
    print(graph.get_graph().draw_ascii())  # fallback ASCII graph

# --- Run Example ---
config = {}  # no thread_id needed since no memory
user_input = "Summarize the latest news about LangGraph"

events = graph.stream({"messages": [("user", user_input)]}, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


/tmp/ipykernel_1744179/2799537363.py:20: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error